In [1]:
# Import Statements
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub = rospy.Publisher('updated_map', Map, queue_size=10)

occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0

map_height = 0
camera_range = 85/map_resolution
overlay = False
display_robot_yaw = True
display_rect = True
display_map = True
display_info = False
tri_width = 5
dist_error = 0.15

np.set_printoptions(threshold=sys.maxsize)

In [3]:
def createLineIterator(P1, P2, img):
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3), dtype=np.int32)
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    return itbuffer

In [4]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [5]:
def disp_map(map_img):
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [6]:
def transform(img):
    
    global bearings
    global areas
    global tags    
    
    bridge = cv_bridge.CvBridge()
    
    areas = []
    bearings = []
    tags = []
    
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    
    
    image = pp(img,width,height)
    
    img2= dbscan(image, img)
    
    return img2
def pp(image, width, height):
    
    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)

    return(image)
def dbscan(image, original):
    start_time_db = time.time()

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))

    if (len(X) > 0):
        start_time = time.time()
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

            i = 0
            x = 0
            l = []
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
#                     time_color = time.time()
                    clr = color_hsv(np.array(l),image)
#                     print "color ", time.time()-time_color
#                     time_rect = time.time()
                    tb = rectangle(np.array(l),clr,image,original)
#                     print "rect ", time.time()-time_rect
#                     print ""
                    l = []
            original = tb
#     print time.time()-start_time_db
#     print "\n"
    return original
def color_hsv(mat, image):
    
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)

    return None

In [7]:
def rectangle(mat,color,image,original):
    
    global bearings
    global overlay
    global occ_map
    global tags
    global display_info
    global map_resolution
    global areas
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
#             print("Area: " + str(area) + " and Tag: " + str(tag) +
#                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(area,-0.4705300238)))
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
#             print("Area: " + str(area) + " and Tag: " + str(tag) +
#                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(area,-0.4705300238)))
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        return image

In [8]:
def edgeFunction(a,b,c):
    det = (c[1]-a[1]) * (b[0]-a[0]) - (c[0]-a[0]) * (b[1]-a[1])
    return det

In [9]:
def rasterizeTri(a,b,c,region):
################################################################
# Rasterize a triangle given three points, a region to search,
#  and a dictionary defining map colors. All points are in (x,y)
#  format
#
# Parameters:
# - a,b: 2 points defining the far vertices of the triangle
# - c: The point where the WAM-V is in occ_map
# - d: The point where the WAM-V is in region
# - region: a portion of the occ_map where the triangle may be
# - values: a dict with the point values defining red and green
#     buoys
#
# Returns:
# - new_region: a region color-coded to highlight the triangle
#     and any buoys in the rasterized region
################################################################
    #print(region)
    region[region < 0] = 0
    #print region
    mask = np.array(np.zeros_like(region.copy()))
    updated = cv.fillPoly(mask, [a,b,c], 255)
    print updated
    
    rast_list = []
    start_time = time.time()
    for i in range(int(min(a[0],b[0],c[0])),int(max(a[0],b[0],c[0]))):
        for j in range(int(min(a[1],b[1],c[1])),int(max(a[1],b[1],c[1]))):
            w0 = edgeFunction(b,c,(i,j))
            w1 = edgeFunction(c,a,(i,j))
            w2 = edgeFunction(a,b,(i,j))

            if w0 >= 0 and w1 >= 0 and w2 >= 0:
                rast_list.append([(i),(j),region[i][j]])
#                 if region[i][j] == values.get("green"):
#                     new_region[i][j] = 255
#                 elif region[i][j] == values.get("red"):
#                     new_region[i][j] = 180
#                 else:
#                     new_region[i][j] = 100
    print time.time()-start_time
    return rast_list

In [10]:
def draw_background_objects(map_img,map_np):
    
    global origin
    global robot_x
    global robot_y
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    map_img[map_np == -1] = [255,255,255]    #Change to [255,255,255] for color to [255] for grayscale
    map_img[map_np == 0] = [255,255,255]     #Change to [255,255,255] for color to [255] for grayscale
    map_img[map_np == 100] = [0,0,0]#Change to [0,0,0] for color to [0] for grayscale
    origin_color = [0,0,255]#Change to [0,0,255] for color to [0] for grayscale
    robot_color = [0,0,255]#Change to [0,0,255] for color to [0] for grayscale
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
    
    return map_img

In [11]:
def draw_ray(map_img,slices):
    
    global display_robot_yaw
    global bearings
    global tags
    global areas
    global robot_x
    global robot_y
    global map_resolution
    global areas
    
    slices_size = len(slices)
    
    if not(display_robot_yaw):
        for i in range(int(slices_size)):
            try:
                arr = slices[i]
                appr_dist_low = (1.0-2.5*dist_error)*map_resolution*2*456.7946397925*math.pow(areas[i],-0.4705300238)
#############################
#                 print (arr.size)
#                 if (arr[:,4] > appr_dist_low):
#                     if (ray[2] == -1 or ray[2] == 0):
#                         map_img[ray[1]][ray[0]] = [255,0,0]
#                     elif ray[2] == 100:
#                         if tags[i] == 2: #here
#                             map_img[ray[1]][ray[0]]=[0,255,0]#Change to [0,255,0] for color to [180 for grayscale]
# #                             print("Distance from buoy: " + str(map_resolution*math.sqrt(math.pow((ray[1]-robot_y),2)+math.pow((ray[0]-robot_x),2))) 
# #                                   + "Tag: green"
# #                                   + "Area: " + str(areas[i]) +
# #                                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(areas[i],-0.4705300238)))
# #                             print ("")
#                             found = True
#                         if tags[i] == 3: #and here
#                             map_img[ray[1]][ray[0]]=[0,0,255]#Change to [0,0,255] for color to [230] for grayscale
# #                             print("Distance from buoy: " + str(map_resolution*math.sqrt(math.pow((ray[1]-robot_y),2)+math.pow((ray[0]-robot_x),2))) 
# #                                   + " Tag: red"
# #                                   + "Area: " + str(areas[i]) +
# #                                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(areas[i],-0.4705300238)))
# #                             print ("")
#                             found = True
####################
                for ray in arr:
                    if ray[3] > appr_dist_low: 
                        if (ray[2] == -1 or ray[2] == 0):
                            if True: #display true if you want to see slice
                                map_img[ray[1]][ray[0]] = [255,0,0]#Change to [255,0,0] for color to [0] for grayscale
                        if ray[2] == 100:
                            if tags[i] == 2:
                                map_img[ray[1]][ray[0]]=[0,255,0]#Change to [0,255,0] for color to [180 for grayscale]
                            if tags[i] == 3: 
                                map_img[ray[1]][ray[0]]=[0,0,255]#Change to [0,0,255] for color to [230] for grayscale
            except Exception as ex:
                print ex
    else:
        for arr in slices[-1]:
            map_img[arr[1]][arr[0]] = [0,255,0]
    
    return map_img

In [12]:
def get_mapnbot_info(trans,robot_yaw):

    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    global map_np
    
    map_array = occ_map.data

    map_width = occ_map.info.width
    map_height = occ_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
    
    return map_np, robot_yaw

In [13]:
def insidetriangle((x1,x2,x3),(y1,y2,y3),map_np):

    global robot_x
    global robot_y
    
    xs=np.array((x1,x2,x3),dtype=float)
    ys=np.array((y1,y2,y3),dtype=float)

    # The possible range of coordinates that can be returned
    x_range=np.arange(np.min(xs),np.max(xs)+1)
    y_range=np.arange(np.min(ys),np.max(ys)+1)

    # Set the grid of coordinates on which the triangle lies. The centre of the
    # triangle serves as a criterion for what is inside or outside the triangle.
    X,Y=np.meshgrid( x_range,y_range )
    xc=np.mean(xs)
    yc=np.mean(ys)

    # From the array 'triangle', points that lie outside the triangle will be
    # set to 'False'.
    triangle = np.ones(X.shape,dtype=bool)
    for i in range(3):
        ii=(i+1)%3
        if xs[i]==xs[ii]:
            include = X *(xc-xs[i])/abs(xc-xs[i]) > xs[i] *(xc-xs[i])/abs(xc-xs[i])
        else:
            poly=np.poly1d([(ys[ii]-ys[i])/(xs[ii]-xs[i]),ys[i]-xs[i]*(ys[ii]-ys[i])/(xs[ii]-xs[i])])
            include = Y *(yc-poly(xc))/abs(yc-poly(xc)) > poly(X) *(yc-poly(xc))/abs(yc-poly(xc))
        triangle*=include

    # Output: 2 arrays with the x- and y- coordinates of the points inside the
    # triangle.
    

    x_points = X[triangle]
    y_points = Y[triangle]

    points = np.column_stack((x_points.astype(int), y_points.astype(int)))

    itbuffer = np.empty(shape=((points.size/2),4), dtype=np.int32)
    itbuffer.fill(np.nan)

    itbuffer[:,0] = x_points
    itbuffer[:,1] = y_points
    itbuffer[:,2] = map_np[points[:,1],points[:,0]]

# Get distance values    

    deltax = itbuffer[:,0]-robot_x
    deltay = itbuffer[:,1]-robot_y
    
    deltax = map_resolution*deltax
    deltay = map_resolution*deltay
    
    
    deltax = np.square(deltax)
    deltay = np.square(deltay)

    
    dist_sq = deltax+deltay
    
    dist_sq = np.sqrt(dist_sq)
#     dist_sq = map_resolution*dist_sq
    
    
    itbuffer[:,3] = dist_sq
#     print robot_x,robot_y
#     print np.column_stack((deltax.astype(int),deltay.astype(int)))
    return itbuffer

In [14]:
def get_points(buoy_yaw, index):
    
    global robot_x
    global robot_y
    global camera_range
    global tri_width
    global areas
    global dist_error
    
    appr_dist_up = (1.0+dist_error)*2*456.7946397925*math.pow(areas[index],-0.4705300238)
    point_deltax = camera_range * math.sin(math.radians(buoy_yaw)) # appr_dist_up
    point_deltay = camera_range * math.cos(math.radians(buoy_yaw)) #appr_dist_up
    per_slope = -1.0/(point_deltay/point_deltax)
    point1 = np.array([round(robot_x), round(robot_y)], dtype=int)
    point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)], dtype=int)

    
    
    point3_x = (robot_x+point_deltax) + math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
    point4_x = (robot_x+point_deltax) - math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))

    point3_y = robot_y+point_deltay + per_slope*(point3_x-(robot_x+point_deltax))
    point4_y = robot_y+point_deltay + per_slope*(point4_x-(robot_x+point_deltax))

    point3 = np.array([round(point3_x),round(point3_y)], dtype=int)
    point4 = np.array([round(point4_x),round(point4_y)], dtype=int)
    
    
    return point1,point2,point3,point4

In [15]:
def map_filter(trans, robot_yaw):
    
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    

    
    slices = []
    i = 0
    while (i < len(bearings)):
        
        
        buoy_yaw = robot_yaw + bearings[i]
        if ((buoy_yaw) < 0):
            buoy_yaw = buoy_yaw + 360
#         print bearings
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
        if display_robot_yaw and occ_map.header.seq % 5 == 0:
            bearings.append(0.0)
            print "Robot_yaw: ", robot_yaw
            print "Buoy yaw: ", buoy_yaw
            print len(slices)
##############################################################################
        
        pt1, pt2, pt3, pt4 = get_points(buoy_yaw, i)    
        
        sliced = createLineIterator(pt1.astype(int),pt2.astype(int),map_np)

#         sliced = insidetriangle((pt3[0],pt1[0],pt4[0]),(pt3[1],pt1[1],pt4[1]),map_np)
        slices.append(np.array(sliced).astype(int))

        i+=1
    
    return slices

In [16]:
def get_map(slices):
    
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))

    #Set to 1 when publishing, to 3 for visualization
    
    map_img_z = np.zeros((map_height, map_width, 3))
    
    map_img_b = draw_background_objects(map_img_z,map_np)
    
    if False:
        map_img_b = np.swapaxes(map_img_b, 0, 1)
        cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
        cv.imshow("map_inverted", map_img_b)
        cv.waitKey(1)
    
    map_img_r = draw_ray(map_img_b,slices)
    
    map_img = np.swapaxes(map_img_r, 0, 1)
    
    return map_img

In [17]:
def callback(rect):
#     start_time = time.time()
    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub
    global map_height
    global map_width
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    
    if not rospy.is_shutdown():       
        try:
            now = rospy.Time.now()
            listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(0.5))
            (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
            pixel_lines = map_filter(trans, math.degrees(euler[2]))
            map_image = get_map(pixel_lines)
#             print (time.time()-start_time)
            if True: #set to false if you don't want to publush the updated occupancy map
#                 up_occ_map_data = np.reshape(np.array(map_image),((map_width*map_height),1))
                pub.publish(up_occ_map)
                
            if display_rect == True:
                disp_rect(rect)
            if display_map == True:
                disp_map(map_image)
        except tf.LookupException as l:
            print l
        except tf.ConnectivityException as c:
            print c
        except tf.ExtrapolationException as e:
            print e
        except Exception as ex:
            print ex 

In [18]:
def map_callback(p_map):
    
    global occ_map
    global up_occ_map
    
    occ_map = p_map    
    up_occ_map = p_map

In [ ]:
if __name__ == '__main__':
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()   

Robot_yaw:  266.786756578
Buoy yaw:  262.255506578
0
Robot_yaw:  266.786756578
Buoy yaw:  248.661756578
1
Robot_yaw:  266.786756578
Buoy yaw:  297.333387438
2
Robot_yaw:  266.786756578
Buoy yaw:  246.520994249
3
Robot_yaw:  266.786756578
Buoy yaw:  266.786756578
4
list index out of range
Robot_yaw:  266.759917788
Buoy yaw:  262.197417788
0
Robot_yaw:  266.759917788
Buoy yaw:  248.634917788
1
Robot_yaw:  266.759917788
Buoy yaw:  297.220244936
2
Robot_yaw:  266.759917788
Buoy yaw:  246.666167788
3
Robot_yaw:  266.759917788
Buoy yaw:  266.759917788
4
list index out of range
Robot_yaw:  266.704087101
Buoy yaw:  262.141587101
0
Robot_yaw:  266.704087101
Buoy yaw:  248.519175373
1
Robot_yaw:  266.704087101
Buoy yaw:  297.416931949
2
Robot_yaw:  266.704087101
Buoy yaw:  246.547837101
3
Robot_yaw:  266.704087101
Buoy yaw:  266.704087101
4
list index out of range
Robot_yaw:  266.668891629
Buoy yaw:  262.106391629
0
Robot_yaw:  266.668891629
Buoy yaw:  248.533548078
1
Robot_yaw:  266.668891629
B

Robot_yaw:  266.745235471
Buoy yaw:  262.307735471
0
Robot_yaw:  266.745235471
Buoy yaw:  248.65550807
1
Robot_yaw:  266.745235471
Buoy yaw:  298.475116758
2
Robot_yaw:  266.745235471
Buoy yaw:  245.429306058
3
Robot_yaw:  266.745235471
Buoy yaw:  266.745235471
4
list index out of range
Robot_yaw:  266.754805389
Buoy yaw:  262.317305389
0
Robot_yaw:  266.754805389
Buoy yaw:  248.692305389
1
Robot_yaw:  266.754805389
Buoy yaw:  298.664030853
2
Robot_yaw:  266.754805389
Buoy yaw:  246.333901229
3
Robot_yaw:  266.754805389
Buoy yaw:  266.754805389
4
list index out of range
Robot_yaw:  266.703743226
Buoy yaw:  262.266243226
0
Robot_yaw:  266.703743226
Buoy yaw:  248.609993226
1
Robot_yaw:  266.703743226
Buoy yaw:  298.766777284
2
Robot_yaw:  266.703743226
Buoy yaw:  246.299707658
3
Robot_yaw:  266.703743226
Buoy yaw:  266.703743226
4
list index out of range
Robot_yaw:  266.764438402
Buoy yaw:  262.358188402
0
Robot_yaw:  266.764438402
Buoy yaw:  248.670688402
1
Robot_yaw:  266.764438402
Bu

Robot_yaw:  266.661420104
Buoy yaw:  262.286420104
0
list index out of range
Robot_yaw:  266.720623608
Buoy yaw:  262.345623608
0
Robot_yaw:  266.720623608
Buoy yaw:  248.658123608
1
Robot_yaw:  266.720623608
Buoy yaw:  299.832401868
2
Robot_yaw:  266.720623608
Buoy yaw:  245.270804044
3
Robot_yaw:  266.720623608
Buoy yaw:  266.720623608
4
list index out of range
Robot_yaw:  266.752893579
Buoy yaw:  262.252893579
0
Robot_yaw:  266.752893579
Buoy yaw:  248.659143579
1
Robot_yaw:  266.752893579
Buoy yaw:  299.895189417
2
Robot_yaw:  266.752893579
Buoy yaw:  246.256782663
3
Robot_yaw:  266.752893579
Buoy yaw:  266.752893579
4
list index out of range
Robot_yaw:  266.653749609
Buoy yaw:  262.170961523
0
Robot_yaw:  266.653749609
Buoy yaw:  248.434999609
1
Robot_yaw:  266.653749609
Buoy yaw:  300.745287084
2
Robot_yaw:  266.653749609
Buoy yaw:  245.913660192
3
Robot_yaw:  266.653749609
Buoy yaw:  266.653749609
4
list index out of range
Robot_yaw:  266.691626311
Buoy yaw:  262.660376311
0
Rob

Robot_yaw:  251.078882216
Buoy yaw:  260.735132216
0
Robot_yaw:  251.078882216
Buoy yaw:  248.078882216
1
Robot_yaw:  251.078882216
Buoy yaw:  249.747262953
2
Robot_yaw:  251.078882216
Buoy yaw:  251.078882216
3
list index out of range
Robot_yaw:  251.098840159
Buoy yaw:  260.817590159
0
Robot_yaw:  251.098840159
Buoy yaw:  248.098840159
1
Robot_yaw:  251.098840159
Buoy yaw:  249.942590159
2
Robot_yaw:  251.098840159
Buoy yaw:  251.098840159
3
list index out of range
Robot_yaw:  251.047402851
Buoy yaw:  260.766152851
0
Robot_yaw:  251.047402851
Buoy yaw:  248.078652851
1
Robot_yaw:  251.047402851
Buoy yaw:  249.922402851
2
Robot_yaw:  251.047402851
Buoy yaw:  251.047402851
3
list index out of range
Robot_yaw:  251.042668238
Buoy yaw:  260.761418238
0
Robot_yaw:  251.042668238
Buoy yaw:  248.042668238
1
Robot_yaw:  251.042668238
Buoy yaw:  249.948918238
2
Robot_yaw:  251.042668238
Buoy yaw:  251.042668238
3
list index out of range
Robot_yaw:  251.061234621
Buoy yaw:  260.779984621
0
Rob

Robot_yaw:  251.128599049
Buoy yaw:  260.998678089
0
Robot_yaw:  251.128599049
Buoy yaw:  248.222349049
1
Robot_yaw:  251.128599049
Buoy yaw:  250.034849049
2
Robot_yaw:  251.128599049
Buoy yaw:  251.128599049
3
list index out of range
Robot_yaw:  251.110221539
Buoy yaw:  250.196651135
2
Robot_yaw:  251.110221539
Buoy yaw:  251.110221539
3
list index out of range
Robot_yaw:  251.090275527
Buoy yaw:  260.902775527
0
Robot_yaw:  251.090275527
Buoy yaw:  248.121525527
1
Robot_yaw:  251.090275527
Buoy yaw:  249.996525527
2
Robot_yaw:  251.090275527
Buoy yaw:  251.090275527
3
list index out of range
Robot_yaw:  251.07509762
Buoy yaw:  260.85634762
0
list index out of range
Robot_yaw:  251.070048383
Buoy yaw:  260.851298383
0
Robot_yaw:  251.070048383
Buoy yaw:  248.070048383
1
Robot_yaw:  251.070048383
Buoy yaw:  249.976298383
2
Robot_yaw:  251.070048383
Buoy yaw:  251.070048383
3
list index out of range
Robot_yaw:  251.076263666
Buoy yaw:  260.920013666
0
Robot_yaw:  251.076263666
Buoy yaw

...

Robot_yaw:  251.06982533
Buoy yaw:  248.06982533
1
Robot_yaw:  251.06982533
Buoy yaw:  249.866868177
2
Robot_yaw:  251.06982533
Buoy yaw:  251.06982533
3
list index out of range
Robot_yaw:  251.068302104
Buoy yaw:  261.099552104
0
Robot_yaw:  251.068302104
Buoy yaw:  248.037052104
1
Robot_yaw:  251.068302104
Buoy yaw:  250.005802104
2
Robot_yaw:  251.068302104
Buoy yaw:  251.068302104
3
list index out of range
Robot_yaw:  251.114536948
Buoy yaw:  261.174557395
0
Robot_yaw:  251.114536948
Buoy yaw:  248.083286948
1
Robot_yaw:  251.114536948
Buoy yaw:  250.052036948
2
Robot_yaw:  251.114536948
Buoy yaw:  251.114536948
3
list index out of range
Robot_yaw:  251.123070959
Buoy yaw:  261.123070959
0
Robot_yaw:  251.123070959
Buoy yaw:  248.123070959
1
Robot_yaw:  251.123070959
Buoy yaw:  250.060570959
2
Robot_yaw:  251.123070959
Buoy yaw:  251.123070959
3
list index out of range
Robot_yaw:  251.11009688
Buoy yaw:  261.14134688
0
Robot_yaw:  251.11009688
Buoy yaw:  248.23509688
1
list index o

Robot_yaw:  251.094934021
Buoy yaw:  261.282434021
0
Robot_yaw:  251.094934021
Buoy yaw:  248.063684021
1
Robot_yaw:  251.094934021
Buoy yaw:  250.032434021
2
Robot_yaw:  251.094934021
Buoy yaw:  251.094934021
3
list index out of range
Robot_yaw:  251.116456897
Buoy yaw:  261.210206897
0
Robot_yaw:  251.116456897
Buoy yaw:  248.085206897
1
Robot_yaw:  251.116456897
Buoy yaw:  250.053956897
2
Robot_yaw:  251.116456897
Buoy yaw:  251.116456897
3
list index out of range
Robot_yaw:  251.129570056
Buoy yaw:  261.223320056
0
Robot_yaw:  251.129570056
Buoy yaw:  248.098320056
1
Robot_yaw:  251.129570056
Buoy yaw:  250.067070056
2
Robot_yaw:  251.129570056
Buoy yaw:  251.129570056
3
list index out of range
Robot_yaw:  251.089512896
Buoy yaw:  261.183262896
0
Robot_yaw:  251.089512896
Buoy yaw:  248.058262896
1
Robot_yaw:  251.089512896
Buoy yaw:  250.027012896
2
Robot_yaw:  251.089512896
Buoy yaw:  251.089512896
3
list index out of range
Robot_yaw:  251.059657731
Buoy yaw:  261.215907731
0
Rob

Robot_yaw:  251.062961243
Buoy yaw:  261.250461243
0
Robot_yaw:  251.062961243
Buoy yaw:  248.031711243
1
Robot_yaw:  251.062961243
Buoy yaw:  250.148727082
2
Robot_yaw:  251.062961243
Buoy yaw:  251.062961243
3
list index out of range
Robot_yaw:  251.058406894
Buoy yaw:  261.277156894
0
Robot_yaw:  251.058406894
Buoy yaw:  248.027156894
1
Robot_yaw:  251.058406894
Buoy yaw:  250.027156894
2
Robot_yaw:  251.058406894
Buoy yaw:  251.058406894
3
list index out of range
Robot_yaw:  251.076835406
Buoy yaw:  261.295585406
0
Robot_yaw:  251.076835406
Buoy yaw:  248.045585406
1
Robot_yaw:  251.076835406
Buoy yaw:  250.045585406
2
Robot_yaw:  251.076835406
Buoy yaw:  251.076835406
3
list index out of range
Robot_yaw:  251.025926551
Buoy yaw:  261.307176551
0
Robot_yaw:  251.025926551
Buoy yaw:  247.963426551
1
Robot_yaw:  251.025926551
Buoy yaw:  249.994676551
2
Robot_yaw:  251.025926551
Buoy yaw:  251.025926551
3
list index out of range
Robot_yaw:  251.063898327
Buoy yaw:  261.282648327
0
Rob

Robot_yaw:  251.051820367
Buoy yaw:  261.458070367
0
Robot_yaw:  251.051820367
Buoy yaw:  247.989320367
1
Robot_yaw:  251.051820367
Buoy yaw:  250.051820367
2
Robot_yaw:  251.051820367
Buoy yaw:  251.051820367
3
list index out of range
Robot_yaw:  251.090050409
Buoy yaw:  261.465050409
0
Robot_yaw:  251.090050409
Buoy yaw:  248.058800409
1
Robot_yaw:  251.090050409
Buoy yaw:  250.090050409
2
Robot_yaw:  251.090050409
Buoy yaw:  251.090050409
3
list index out of range
Robot_yaw:  251.115505895
Buoy yaw:  248.084255895
1
Robot_yaw:  251.115505895
Buoy yaw:  250.115505895
2
Robot_yaw:  251.115505895
Buoy yaw:  251.115505895
3
list index out of range
Robot_yaw:  251.124488556
Buoy yaw:  248.093238556
1
Robot_yaw:  251.124488556
Buoy yaw:  250.093238556
2
Robot_yaw:  251.124488556
Buoy yaw:  251.124488556
3
list index out of range
Robot_yaw:  251.08525707
Buoy yaw:  261.42900707
0
Robot_yaw:  251.08525707
Buoy yaw:  248.05400707
1
Robot_yaw:  251.08525707
Buoy yaw:  250.11650707
2
Robot_yaw

Robot_yaw:  251.064714071
Buoy yaw:  261.502214071
0
Robot_yaw:  251.064714071
Buoy yaw:  248.002214071
1
Robot_yaw:  251.064714071
Buoy yaw:  250.064714071
2
Robot_yaw:  251.064714071
Buoy yaw:  251.064714071
3
list index out of range
Robot_yaw:  251.018071053
Buoy yaw:  261.486821053
0
Robot_yaw:  251.018071053
Buoy yaw:  247.955571053
1
Robot_yaw:  251.018071053
Buoy yaw:  249.986821053
2
Robot_yaw:  251.018071053
Buoy yaw:  251.018071053
3
list index out of range
Robot_yaw:  251.028184297
Buoy yaw:  261.496934297
0
Robot_yaw:  251.028184297
Buoy yaw:  247.965684297
1
Robot_yaw:  251.028184297
Buoy yaw:  249.908666017
2
Robot_yaw:  251.028184297
Buoy yaw:  251.028184297
3
list index out of range
Robot_yaw:  251.062401715
Buoy yaw:  261.499901715
0
Robot_yaw:  251.062401715
Buoy yaw:  247.999901715
1
Robot_yaw:  251.062401715
Buoy yaw:  250.267147961
2
Robot_yaw:  251.062401715
Buoy yaw:  251.062401715
3
list index out of range
Robot_yaw:  251.097338025
Buoy yaw:  261.534838025
0
lis